# Cluster Finder - Example Usage

This notebook demonstrates the basic functionality of the `cluster_finder` package for finding, analyzing, and visualizing atomic clusters in crystal structures.

## Setup

First, let's import the necessary libraries and set up our environment.

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from pymatgen.core.structure import Structure
from pymatgen.core.lattice import Lattice

# Add parent directory to path so we can import the package
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname('..'), '..')))

import cluster_finder as cf

# Configure matplotlib for better visualization
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 10]
plt.rcParams['figure.dpi'] = 100

## 1. Create a Simple Structure

Let's create a simple cubic structure with Fe atoms to work with.

In [ ]:
def create_simple_structure():
    """Create a simple cubic structure with Fe atoms."""
    # Create a simple cubic lattice with Fe atoms
    lattice = Lattice.cubic(3.0)  # 3 Å cube
    
    # Create a structure with Fe atoms at corners
    structure = Structure(
        lattice=lattice,
        species=['Fe', 'Fe', 'Fe', 'Fe', 'Fe', 'Fe', 'Fe', 'Fe'],
        coords=[
            [0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0],
            [0, 0, 1], [1, 0, 1], [0, 1, 1], [1, 1, 1]
        ],
        coords_are_cartesian=False
    )
    
    return structure

# Create our structure
structure = create_simple_structure()
print(f"Created structure with {len(structure)} sites")
print(f"Formula: {structure.composition.reduced_formula}")

## 2. Find Clusters

Now let's find clusters in our structure using the `cluster_finder` package.

In [ ]:
# Define transition metals to search for
transition_metals = cf.get_transition_metals()
print(f"Transition metals (first 5): {', '.join(transition_metals[:5])}...")

# Create connectivity matrix
connectivity_matrix, tm_indices = cf.create_connectivity_matrix(
    structure, 
    transition_metals,
    cutoff=3.5  # Distance cutoff for connectivity
)

print(f"Connectivity matrix shape: {connectivity_matrix.shape}")
print(f"Number of transition metal sites: {len(tm_indices)}")

# Create graph and find clusters
graph = cf.structure_to_graph(connectivity_matrix)
print(f"Graph has {graph.number_of_nodes()} nodes and {graph.number_of_edges()} edges")

# Find clusters with minimum size 2
clusters = cf.find_clusters(structure, graph, tm_indices, min_cluster_size=2)
print(f"Found {len(clusters)} clusters with minimum size 2")

## 3. Analyze Clusters

Let's analyze the clusters we found.

In [ ]:
# Analyze clusters
analyzed_clusters = cf.analyze_clusters(clusters, structure.lattice)

print("Cluster details:")
for i, cluster in enumerate(analyzed_clusters):
    elements = [site.specie.symbol for site in cluster["sites"]]
    print(f"  Cluster {i+1}: {len(elements)} atoms ({', '.join(elements)}), "
          f"avg. distance: {cluster['average_distance']:.3f} Å")

# Classify structure dimensionality
dimensionality = cf.classify_dimensionality(structure)
print(f"\nStructure dimensionality: {dimensionality}")

## 4. Visualize Clusters

Let's visualize the clusters and connectivity in our structure.

In [ ]:
# Visualize connectivity graph
plt.figure(figsize=(12, 10))
fig1 = cf.visualize_graph(
    graph, 
    structure, 
    tm_indices, 
    material_id="example", 
    formula=structure.composition.reduced_formula
)
plt.title("Connectivity Graph", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Visualize clusters in the compound
if analyzed_clusters:
    plt.figure(figsize=(12, 10))
    fig2 = cf.visualize_clusters_in_compound(structure, analyzed_clusters)
    plt.title("Clusters in Structure", fontsize=16)
    plt.tight_layout()
    plt.show()

## 5. Rotate and Visualize Structure

Let's visualize the structure with different rotations.

In [ ]:
# Create a rotation matrix (45° around y-axis)
rotation_matrix = cf.generate_rotation_matrix(
    axis=np.array([0, 1, 0]),
    angle=np.pi/4
)

plt.figure(figsize=(12, 10))
fig3 = cf.visualize_cluster_lattice(structure, rotation_matrix)
plt.title("Rotated Structure View", fontsize=16)
plt.tight_layout()
plt.show()

## 6. Export Results

Finally, let's export our results to various file formats.

In [ ]:
# Create output directory
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

# Export structure to CIF
cif_path = os.path.join(output_dir, "structure.cif")
cf.export_structure_to_cif(structure, cif_path)
print(f"Exported structure to {cif_path}")

# Generate lattice with clusters
if analyzed_clusters:
    cluster_structure = cf.generate_lattice_with_clusters(structure, analyzed_clusters)
    cluster_cif_path = os.path.join(output_dir, "cluster_centroids.cif")
    cf.export_structure_to_cif(cluster_structure, cluster_cif_path)
    print(f"Exported cluster centroids structure to {cluster_cif_path}")

## Conclusion

This notebook demonstrated the basic functionality of the `cluster_finder` package. We:
1. Created a simple structure
2. Found and analyzed clusters
3. Visualized the clusters and connectivity
4. Exported the results to files

For more advanced functionality, refer to the comprehensive `example_usage.py` script and the package documentation.